In [2]:
try:
    import tensorflow as tf
except ImportError:
    import sys
    !{sys.executable} -m pip install tensorflow
    import tensorflow as tf
    
import numpy as np
from tensorflow.keras import layers, models
from sklearn.model_selection import train_test_split
import json
import os

2025-05-25 14:41:00.540461: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-25 14:41:01.368987: I external/local_xla/xla/tsl/cuda/cudart_stub.cc:32] Could not find cuda drivers on your machine, GPU will not be used.
2025-05-25 14:41:02.422594: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1748184063.105881   23348 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1748184063.286202   23348 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1748184064.388689   23348 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linkin

In [ ]:
tf.config.set_visible_devices([], 'GPU')

# 2. Cargar datos
images = np.load('images.npy')
categories = np.load('categories.npy')

# 3. Dividir en train / temp (para luego val/test)
X_train, X_temp, y_train, y_temp = train_test_split(images, categories, test_size=0.25, stratify=categories, random_state=42)

# 4. Dividir temp en validación y test
X_val, X_test, y_val, y_test = train_test_split(X_temp, y_temp, test_size=0.2, stratify=y_temp, random_state=42)
# Resultado: 75% train, 20% val, 5% test

# 5. One-hot encoding (porque usaremos categorical_crossentropy)
num_classes = 6
y_train_cat = tf.keras.utils.to_categorical(y_train, num_classes)
y_val_cat = tf.keras.utils.to_categorical(y_val, num_classes)
y_test_cat = tf.keras.utils.to_categorical(y_test, num_classes)

# 6. Definir el modelo base CNN
model = models.Sequential([
    layers.Input(shape=(150, 150, 3)),
    layers.Conv2D(32, (3, 3), activation='relu'),
    layers.MaxPooling2D((2, 2)),

    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(num_classes, activation='softmax')
])

model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# 7. Entrenar con EarlyStopping
early_stop = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

history = model.fit(X_train, y_train_cat,
                    epochs=30,
                    batch_size=32,
                    validation_data=(X_val, y_val_cat),
                    callbacks=[early_stop])

# 8. Guardar modelo .h5
model.save("modelo_base_cnn.h5")
print("Modelo guardado como modelo_base_cnn.h5")

# 9. Guardar history en JSON
history_dict = history.history
with open("history_modelo_base.json", "w") as f:
    json.dump(history_dict, f)
print("History guardado como history_modelo_base.json")

2025-05-25 14:41:26.912295: E external/local_xla/xla/stream_executor/cuda/cuda_platform.cc:51] failed call to cuInit: INTERNAL: CUDA error: Failed call to cuInit: UNKNOWN ERROR (303)
2025-05-25 14:42:55.392140: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 44859392 exceeds 10% of free system memory.
2025-05-25 14:42:55.460263: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 44859392 exceeds 10% of free system memory.
2025-05-25 14:42:55.488101: W external/local_xla/xla/tsl/framework/cpu_allocator_impl.cc:83] Allocation of 44859392 exceeds 10% of free system memory.
